## Purpose

This notebook demonstrates a Calcium Imaging pipeline _workflow_ with an ISX library, including loading videos, running processing stages, and producing traceable outputs. The goal is to help users understand and reproduce end-to-end data analysis pipelines efficiently.

## Dependencies Installation

In [ ]:
import sys


def calculate_project_root_directory_from_notebook_location():
    project_root = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
    if project_root not in sys.path:
        sys.path.insert(0, project_root)

In [ ]:
import isx
from src.ci_pipe import CIPipe

In [ ]:
import os
upper_root = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
requirements_path = os.path.join(upper_root, "requirements.txt")
!pip install -r $requirements_path

## Pipeline minimal creation

Replace 'input_dir' and 'output_dir' with your actual directories


In [ ]:
pipeline = CIPipe.with_videos_from_directory('input_dir', outputs_directory='output_dir', isx=isx)

### Full pipeline execution with ISX algorithms

In [ ]:
(
    pipeline
    .set_defaults(isx_bp_subtract_global_minimum=False, isx_mc_max_translation=25, isx_acr_filters=[('SNR', '>', 3), ('Event Rate', '>', 0), ('# Comps', '=', 1)])
    .isx.preprocess_videos()
    .isx.bandpass_filter_videos()
    .isx.motion_correction_videos(isx_mc_series_name="series1")
    .isx.normalize_dff_videos()
    .isx.extract_neurons_pca_ica()
    .isx.detect_events_in_cells()
    .isx.auto_accept_reject_cells()
    .isx.longitudinal_registration(isx_lr_reference_selection_strategy='by_num_cells_desc')
)